In [1]:
import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

In [2]:
# import sys
# !{sys.executable} -m pip install -U scikit-learn

In [3]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR

from sklearn.metrics import mean_absolute_error as sk_mean_absolute_error
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd

import eli5
from eli5.sklearn import PermutationImportance

class processutil:
    def _str2class(s):
        if s in globals() and isinstance(globals()[s], type):
                return globals()[s]
        if isinstance(eval(s), type):
            return eval(s)
        if callable(eval(s)):
            return eval(s)
        return None
    

Using TensorFlow backend.


In [4]:
folder_path = '../data/IEEE-CIS-Fraud-Detection/'

In [5]:
df_train = pd.read_pickle(f'{folder_path}/df_train2.gzde', compression='gzip')#.iloc[:100000,:]
# df_test = pd.read_pickle(f'{folder_path}/df_test2.gzde', compression='gzip').iloc[:10000,:]

In [6]:
columns = df_train.columns.tolist()
columns.remove('TransactionID')
columns.remove('TransactionDT')
columns.remove('isFraud')

In [7]:
len(columns)

557

In [8]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
         }

In [9]:
from imblearn.over_sampling import SMOTE

# smote = SMOTE(ratio='minority')
# X_sm, y_sm = smote.fit_sample(X, y)

# plot_2d_space(X_sm, y_sm,X,y, 'SMOTE over-sampling')

In [10]:
from imblearn.under_sampling import RandomUnderSampler

# ran=RandomUnderSampler(return_indices=True) ##intialize to return indices of dropped rows
# X_rs,y_rs,dropped = ran.fit_sample(X,y)

In [21]:
def process(folds, df_train,df_test, columns, is_output_feature_importance=1, verbose=0):

#     aucs = list()
    his = []
    training_start_time = time()
    df_feature_importances_i_list = []
    df_valid_pred = pd.DataFrame()
    df_test_pred = pd.DataFrame()
    if type(df_test) != type(None):
        df_test_pred['TransactionID'] = df_test['TransactionID']
    
    X,y = df_train.sort_values('TransactionDT')[columns], df_train.sort_values('TransactionDT')['isFraud']
    if type(df_test) != type(None):
        X_test = df_test.sort_values('TransactionDT')[columns]
        
    for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
        start_time = time()
        if verbose > 1:
            print('Training on fold {}'.format(fold + 1))


#         smote = SMOTE(ratio='minority')
#         X_train_fold_n, y_train_fold_n = smote.fit_sample(X.iloc[trn_idx], y.iloc[trn_idx])
#         ran=RandomUnderSampler(return_indices=True) 
#         X_train_fold_n, y_train_fold_n,dropped = ran.fit_sample(X.iloc[trn_idx], y.iloc[trn_idx])
        
        ss = StandardScaler()
        X_train_fold_n, y_train_fold_n = ss.fit_transform(X.iloc[trn_idx] ), y.iloc[trn_idx]
        X_test_fold_n, y_test_fold_n = ss.transform(X.iloc[test_idx]), y.iloc[test_idx]
        
        trn_data = lgb.Dataset(X_train_fold_n, label=y_train_fold_n)
        val_data = lgb.Dataset(X_test_fold_n, label=y_test_fold_n)
        clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=500)
        
        y_trn_pred = clf.predict(X.iloc[trn_idx].values)
        y_val_pred = clf.predict(X.iloc[test_idx].values)
        
        original_index = df_train['TransactionID'].values[test_idx]
        df_valid_pred_i = pd.DataFrame({'TransactionID': original_index, 'predict': y_val_pred, 'fold': np.zeros(y_val_pred.shape[0]) + fold})
        df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
        
        y_test_pred = None
        if type(df_test)!=type(None):
            y_test_pred = clf.predict(X_test.values)
            df_test_pred_i = pd.DataFrame({fold: y_test_pred})
            df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=1)
        
        trn_auc = roc_auc_score(y.iloc[trn_idx].values, y_trn_pred)
        val_auc = roc_auc_score(y.iloc[test_idx].values, y_val_pred)
        
        his.append({'val_auc':val_auc, 'trn_auc':trn_auc, 'y_val_pred':y_val_pred, 'y_test_pred':y_test_pred, 'test_idx':test_idx})
        
        if is_output_feature_importance:
            best_iter = clf.best_iteration
            clf = lgb.LGBMClassifier(**params, num_boost_round=best_iter)
            clf.fit(X.iloc[trn_idx].values, y.iloc[trn_idx].values)
            perm = PermutationImportance(clf, random_state=42).fit(X.iloc[test_idx].values, y.iloc[test_idx].values)
            df_feature_importances_i2 = eli5.explain_weights_dfs(perm, feature_names=columns, top=len(columns))['feature_importances']
            df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['feature'])
            df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)
            df_feature_importances_i_list.append(df_feature_importances_i2)
        
#         aucs.append(clf.best_score['valid_1']['auc'])
#         his.append({'val_auc':val_auc, 'trn_auc':trn_auc, 'y_val_pred':y_val_pred, 'y_test_pred':y_test_pred, 'test_idx':test_idx})
        if verbose > 0:
            print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
    
    his = pd.DataFrame(his)
    
    df_feature_importances = None
    if is_output_feature_importance:
        df_feature_importances = df_feature_importances_i_list[0]
        for idx, df_feature_importances_i in enumerate(df_feature_importances_i_list[1:]):
            df_feature_importances = pd.merge(df_feature_importances, df_feature_importances_i, on='feature', suffixes=('', idx + 1))
            
    df_valid_pred = df_valid_pred.sort_values(by=['TransactionID'])
    df_valid_pred = df_valid_pred.reset_index(drop=True)

    if type(df_test) != type(None):
        df_test_pred = df_test_pred.sort_values(by=['TransactionID'])
        df_test_pred = df_test_pred.reset_index(drop=True)
    
    if verbose > 0:
        print('-' * 30)
        print('Training has finished.')
        print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
        print('Mean AUC:', his.val_auc.mean(), his.trn_auc.mean())
        print('-' * 30)
    return his, df_feature_importances, df_valid_pred, df_test_pred, his.val_auc.mean()

In [24]:
# folds = TimeSeriesSplit(n_splits=5)

folds = KFold(n_splits=8, shuffle=False, random_state=42)

In [ ]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(folds, df_train,None, columns, is_output_feature_importance=1, verbose=1)

Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.999006	valid_1's auc: 0.92588
[2000]	training's auc: 0.999999	valid_1's auc: 0.927713
[3000]	training's auc: 1	valid_1's auc: 0.928073
Early stopping, best iteration is:
[2521]	training's auc: 1	valid_1's auc: 0.927938
Fold 1 finished in 4:50:52.627341
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.998934	valid_1's auc: 0.940446
[2000]	training's auc: 0.999999	valid_1's auc: 0.943979
Early stopping, best iteration is:
[2093]	training's auc: 1	valid_1's auc: 0.944104


In [ ]:
Mean AUC: 0.9271172613055931 0.9999956794809108